In [1]:
from pathlib import Path
import sys
import time
import threading

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
%matplotlib inline
import autoclasswrapper as wrapper
import os 

afterall = {}
subfeatures = ['P0', 'P1', 'P2', 'P3', 'P4']
os.environ['PATH'] += ":/home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c"
#!touch /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
#!echo "xref_class_report_att_list = 0, 1 \nreport_mode = \"data\" \ncomment_data_headers_p = true " >> /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
for feature in subfeatures:
    runtimeDir = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round1/" + feature
    if not Path(runtimeDir).exists():
        os.mkdir(runtimeDir)
    os.chdir(runtimeDir)
    
            # Create object to prepare dataset.
    clust = wrapper.Input()

    # Load datasets from tsv files.
    pathData = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/" + feature + ".tsv"
    clust.add_input_data(pathData, "real location")
    #clust.add_input_data("/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P2.tsv", "real location")
    
    # Prepare input data:
    # - create a final dataframe
    # - merge datasets if multiple inputs
    clust.prepare_input_data()

    # Create files needed by AutoClass.
    clust.create_db2_file()
    clust.create_hd2_file()
    clust.create_model_file()
    clust.create_sparams_file(max_duration = 0)
    clust.create_rparams_file()
    !rm autoclass.r-params
    !cp /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round1/autoclass.r-params .
    
    !rm -f autoclass-run-* *.results-bin


    # Search autoclass in path.
    wrapper.search_autoclass_in_path()

    # Create object to run AutoClass.
    run = wrapper.Run()

    # Prepare run script.
    run.create_run_file()

    # Run AutoClass.
    run.run()
        
    timer = 0
    step = 2
    while not Path(runtimeDir + "/autoclass-run-success").exists():
        timer += step
        sys.stdout.write("\r")
        sys.stdout.write(f"Time: {timer} sec.")
        sys.stdout.flush()
        time.sleep(step)
        if Path("autoclass-run-failure").exists():
            print("error")
            break
            
    if Path(runtimeDir + "/autoclass-run-success").exists():
        results = wrapper.Output()
        results.extract_results()
        results.aggregate_input_data()
#         results.write_cdt()
#         results.write_cdt(with_proba=True)
#         results.write_class_stats()
#         results.write_dendrogram()
        result = pd.read_csv(runtimeDir + '/autoclass_out.tsv', encoding="utf-8", header=0, sep='\t', index_col=0)
        label =  pd.read_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/label.tsv', encoding='utf-8', header=0, sep='\t', index_col=0)

        label['cluster'] = result['main-class']
        label.to_csv('app.csv', index = True, header=True)


        listH = []
        listApp = []
        listCluster = []
        def fun(x):
            listCluster.append(x['cluster'].values[0])
            listH.append(x['label'].value_counts().max()/x.shape[0])
            listApp.append(x['label'].value_counts().idxmax())


        label.groupby('cluster').apply(fun)

        H = sum(listH)/len(listH)
        print(listH)
        print(listApp)
        print(listCluster)
        print(H)

        dict = {'Cluster': listCluster, 'App': listApp, 'H': listH}

        saveResult = pd.DataFrame(dict)

        saveResult.to_csv(runtimeDir + '/resultCal.csv', index = True, header=True)
        
        afterall.update({feature:H})
        
round1 = pd.DataFrame(list(afterall.items()), index= range(len(subfeatures)))
round1.info()
round1.head(1)
round1.to_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round1/Hp1.csv', index = True, header=True)

2020-04-27 15:21:57 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P0.tsv' as 'real location' with error 0.01
2020-04-27 15:21:57 INFO     Detected encoding: ascii
2020-04-27 15:21:57 INFO     Found 3200 rows and 2 columns
2020-04-27 15:21:57 DEBUG    Checking column names
2020-04-27 15:21:57 DEBUG    Index name 'index'
2020-04-27 15:21:57 DEBUG    Column name 'P0'
2020-04-27 15:21:57 INFO     Checking data format
2020-04-27 15:21:57 INFO     Column 'P0'
2020-04-27 15:21:57 INFO     count    3.200000e+03
2020-04-27 15:21:57 INFO     mean    -7.765439e+03
2020-04-27 15:21:57 INFO     std      1.036135e+06
2020-04-27 15:21:57 INFO     min     -1.783918e+06
2020-04-27 15:21:57 INFO     50%      6.245000e+03
2020-04-27 15:21:57 INFO     max      1.786032e+06
2020-04-27 15:21:57 INFO     ---
2020-04-27 15:21:57 INFO     Final dataframe has 3200 lines and 2 columns
2020-04-27 15:21:57 INFO     Searching for missing values
2020-04-27 15:21:57

Time: 172 sec.

2020-04-27 15:24:50 INFO     Extracting autoclass results
2020-04-27 15:24:50 INFO     Found 3200 cases classified in 6 classes
2020-04-27 15:24:52 INFO     Aggregating input data
2020-04-27 15:24:52 INFO     Writing classes + probabilities .tsv file
2020-04-27 15:24:52 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P1.tsv' as 'real location' with error 0.01
2020-04-27 15:24:52 INFO     Detected encoding: ascii
2020-04-27 15:24:52 INFO     Found 3200 rows and 2 columns
2020-04-27 15:24:52 DEBUG    Checking column names
2020-04-27 15:24:52 DEBUG    Index name 'index'
2020-04-27 15:24:52 DEBUG    Column name 'P1'
2020-04-27 15:24:52 INFO     Checking data format
2020-04-27 15:24:52 INFO     Column 'P1'
2020-04-27 15:24:52 INFO     count    3200.000000
2020-04-27 15:24:52 INFO     mean       -0.300688
2020-04-27 15:24:52 INFO     std         3.208397
2020-04-27 15:24:52 INFO     min        -3.908736
2020-04-27 15:24:52 INFO     50%       

[0.04992867332382311, 0.08411214953271028, 0.07235142118863049, 0.09953703703703703, 0.08949416342412451, 0.08021390374331551]
['SPOTIFY', 'APPLE', 'MICROSOFT', 'INSTAGRAM', 'INSTAGRAM', 'WIKIPEDIA']
[1, 2, 3, 4, 5, 6]
0.07927289137494016


2020-04-27 15:24:52 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 15:24:52 INFO     Writing run file
2020-04-27 15:24:52 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 15:24:52 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 15:24:52 INFO     Running clustering...


...

[0.05898617511520737, 0.11335012594458438, 0.07153284671532846, 0.1996007984031936, 0.10311750599520383, 0.11009174311926606, 0.3333333333333333]
['DROPBOX', 'DNS', 'HTTP', 'IP_ICMP', 'AMAZON', 'DNS', 'NETFLIX']
[1, 2, 3, 4, 5, 6, 7]
0.14143036123230243


2020-04-27 15:34:36 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 15:34:36 INFO     Writing run file
2020-04-27 15:34:36 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 15:34:36 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 15:34:36 INFO     Running clustering...


Time: 142 sec.

2020-04-27 15:36:58 INFO     Extracting autoclass results
2020-04-27 15:36:58 INFO     Found 3200 cases classified in 5 classes
2020-04-27 15:37:02 INFO     Aggregating input data
2020-04-27 15:37:02 INFO     Writing classes + probabilities .tsv file


[0.059927797833935016, 0.07122093023255814, 0.04957102001906578, 0.2535211267605634, 0.2857142857142857]
['IP_ICMP', 'WINDOWS_UPDATE', 'HTTP', 'OFFICE_365', 'NETFLIX']
[1, 2, 3, 4, 5]
0.14399103211208159
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       5 non-null      object 
 1   1       5 non-null      float64
dtypes: float64(1), object(1)
memory usage: 208.0+ bytes
